<a href="https://colab.research.google.com/github/vfunnny2000/PythonAnalytics_and_A-B_tests/blob/master/A%26B_tests_HW_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Задача

Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

Mobile Games AB Testing with Cookie Cats

In [1]:

from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np
import plotly.express as px

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

In [2]:
from google.colab import files
 
uploaded = files.upload()

Saving gb_sem_9_hw.csv to gb_sem_9_hw.csv


In [3]:
data = pd.read_csv('gb_sem_9_hw.csv')

In [4]:
data

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [6]:
from operator import mul
data.version.replace({'gate_30': 0, 'gate_40': 1}, inplace=True)

In [7]:
data

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,0,3,False,False
1,337,0,38,True,False
2,377,1,165,True,False
3,483,1,1,False,False
4,488,1,179,True,True
...,...,...,...,...,...
90184,9999441,1,97,True,False
90185,9999479,1,30,False,False
90186,9999710,0,28,True,False
90187,9999768,1,51,True,False


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   userid          90189 non-null  int64
 1   version         90189 non-null  int64
 2   sum_gamerounds  90189 non-null  int64
 3   retention_1     90189 non-null  bool 
 4   retention_7     90189 non-null  bool 
dtypes: bool(2), int64(3)
memory usage: 2.2 MB


In [9]:
data.describe()

,userid,version,sum_gamerounds
count,9.018900e+04,90189.000000,90189.000000
mean,4.998412e+06,0.504374,51.872457
std,2.883286e+06,0.499984,195.050858
min,1.160000e+02,0.000000,0.000000
25%,2.512230e+06,0.000000,5.000000
50%,4.995815e+06,1.000000,16.000000
75%,7.496452e+06,1.000000,51.000000
max,9.999861e+06,1.000000,49854.000000


Первоначальный визуальный анализ говорит о предположительно равномерном распределении.

In [10]:
data.version.equals(data.sum_gamerounds)

False

In [11]:
data.version.compare(data.sum_gamerounds).index

Int64Index([    0,     1,     2,     4,     5,     7,     8,     9,    10,
               12,
            ...
            90179, 90180, 90181, 90182, 90183, 90184, 90185, 90186, 90187,
            90188],
           dtype='int64', length=85463)

In [12]:
data.iloc[data.version.compare(data.sum_gamerounds).index, :]

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,0,3,False,False
1,337,0,38,True,False
2,377,1,165,True,False
4,488,1,179,True,True
5,540,1,187,True,True
...,...,...,...,...,...
90184,9999441,1,97,True,False
90185,9999479,1,30,False,False
90186,9999710,0,28,True,False
90187,9999768,1,51,True,False


In [13]:
data_2 = data.drop(data.version.compare(data.sum_gamerounds).index).copy(deep=True)

In [14]:
data_2.version.equals(data_2.sum_gamerounds)

True

In [15]:
data_2.userid.value_counts()

483        1
6618245    1
6643383    1
6642819    1
6642100    1
          ..
3264631    1
3263072    1
3257040    1
3253065    1
9998285    1
Name: userid, Length: 4726, dtype: int64

In [16]:
data_2 = data_2.loc[data_2.userid.isin(data_2.userid.value_counts()[data_2.userid.value_counts() == 1].index.values), :]

In [17]:
data_2

,userid,version,sum_gamerounds,retention_1,retention_7
3,483,1,1,False,False
6,1066,0,0,False,False
11,2101,0,0,False,False
19,2502,1,1,False,False
26,3095,0,0,False,False
...,...,...,...,...,...
90116,9990773,0,0,False,False
90147,9994604,0,0,False,False
90148,9994647,1,1,False,False
90149,9995246,1,1,False,False


In [18]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [19]:
def proportion_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Вероятность событий
    size = control.loc[:, column].shape[0]
    prop_control = control.loc[:, column].sum() / size
    prop_treatment = treatment.loc[:, column].sum() / size
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = stats.bernoulli.rvs(p=prop_control, size=size)
        treatment_sample = stats.bernoulli.rvs(p=prop_treatment, size=size)
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size = proportion.proportion_effectsize(prop_control, prop_treatment)
    
    power = zt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [20]:
control = data[data.sum_gamerounds == 0].copy(deep=True)
treatment = data[data.sum_gamerounds == 1].copy(deep=True)

In [21]:
control.shape, treatment.shape

((3994, 5), (5538, 5))

In [22]:
fig = px.histogram(data,
                   x='sum_gamerounds',
                   color = 'version',
                   title='avg_site_visits_distribution',
                   marginal = 'box',
                   nbins = 100,
                   barmode='overlay')

fig.show()

Выборка небольшая, но сконцентрирована около нуля, достаточно равномерная с небольшим количеством выбросов.

In [23]:
continious_result(control, treatment, 'sum_gamerounds')

100%|██████████| 10000/10000 [00:13<00:00, 757.61it/s]
<ipython-input-18-989b12ce17ca>:24: RuntimeWarning:

divide by zero encountered in double_scalars

/usr/local/lib/python3.9/dist-packages/statsmodels/stats/meta_analysis.py:386: RuntimeWarning:

divide by zero encountered in double_scalars



,effect_size,alpha,beta,CI,difference
sum_gamerounds,inf,0.0,NaN,"[-1.0, -1.0]",-1.0


Мощности теста *недостаточно* для выводов и заключения по эксперименту

## Bucket

In [24]:
for _ in range(100, 1001): 
    if data_2.shape[0] % _ == 0:
        print(_)

139
278


In [25]:
n_buckets = 278
data_2 = (data_2
 .sample(n=data_2.shape[0], replace=False)
 .reset_index(drop=True)
 .assign(bucket=list(range(n_buckets)) * int(data_2.shape[0] / n_buckets)))

In [26]:
data_2.head()

,userid,version,sum_gamerounds,retention_1,retention_7,bucket
0,876954,1,1,False,False,0
1,6167588,0,0,False,False,1
2,830618,1,1,False,False,2
3,8456906,1,1,True,False,3
4,1436875,1,1,False,False,4


In [27]:
bucketed_data_2 = data_2.groupby(['version', 'bucket'])['sum_gamerounds'].agg(mu=np.mean, std=np.std).reset_index()
bucketed_data_2

,version,bucket,mu,std
0,0,0,0.0,0.0
1,0,1,0.0,0.0
2,0,2,0.0,0.0
3,0,3,0.0,0.0
4,0,4,0.0,0.0
...,...,...,...,...
551,1,273,1.0,0.0
552,1,274,1.0,0.0
553,1,275,1.0,0.0
554,1,276,1.0,0.0


In [28]:
# Сравним исходное выборочное среднее и среднее бакетных средних 
round(np.mean(data_2["sum_gamerounds"]), 5), round(np.mean(bucketed_data_2["mu"]), 5)

(0.59014, 0.5)

In [29]:
round(np.std(data_2["sum_gamerounds"]), 5), round(np.mean(bucketed_data_2["std"]), 5)

(0.49181, 0.0)

In [30]:
control_bucket = bucketed_data_2[bucketed_data_2.version == 0]
treatment_bucket = bucketed_data_2[bucketed_data_2.version == 1]
continious_result(control_bucket, treatment_bucket, 'mu')
     

100%|██████████| 10000/10000 [00:03<00:00, 3247.31it/s]
<ipython-input-18-989b12ce17ca>:24: RuntimeWarning:

divide by zero encountered in double_scalars

/usr/local/lib/python3.9/dist-packages/statsmodels/stats/meta_analysis.py:386: RuntimeWarning:

divide by zero encountered in double_scalars



,effect_size,alpha,beta,CI,difference
mu,inf,0.0,NaN,"[-1.0, -1.0]",-1.0


Результат *аналогичный* тому, что был выше

## Testing converted

In [31]:
fig = px.histogram(data_2, x="retention_1",
                   color='version', barmode='group',
                   height=400)
fig.show()
     

In [32]:
proportion_result(control, treatment, 'retention_1')

100%|██████████| 10000/10000 [00:03<00:00, 2816.89it/s]


,effect_size,alpha,beta,CI,difference
retention_1,-0.120582,1.571102e-07,0.000303,"[-0.029, -0.013]",-0.020973


**или**

In [33]:
fig = px.histogram(data_2, x="retention_7",
                   color='version', barmode='group',
                   height=400)
fig.show()

In [34]:
proportion_result(control, treatment, 'retention_7')

100%|██████████| 10000/10000 [00:03<00:00, 2883.42it/s]


,effect_size,alpha,beta,CI,difference
retention_7,-0.032356,0.144589,0.696056,"[-0.007, 0.001]",-0.003024


Можно сделать вывод о преимуществе по скорости тестирования при использовании Bucket и при проведении Testing converted, в результате которого значение альфа и бета больше допустимого порога, получилось значительно выше, ноль в доверительный интервал retention_1 не входит, retention_7 ноль входит. 

**Результаты теста не статстически значимы.** 

Рекомендауется перезапустить тест, чтобы увеличить мощность и соответственно следует увеличить объем выборки, чтобы было достаточное количество данных для полноценного анализа. 